Author: Carlos Roberto de Melo

Date: 06/18/2020
    
Obj: Convert the quantities of MGE output into physical quantities that can be used for the dynamic model (JAM).

In [1]:
import numpy as np

In [7]:
#Some properties of img
#Imagem SDP.81 WFC3/HST

SCALE = 0.09                 #arcsec
EXPTIME = 4417.621094        #seconds
Vega_zeropoint = 24.662      #Vega magnitude zeropoint in the F160w band from (https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration)
M_sun_160 = 3.37             #Magnitude absoluta do Sol com relação a Vega, na banda ACS/F814W from Willmer, Christopher N. A. (2018)
A_160 = 0.018                #Extinction from NED-Caltch

In [8]:
#Lendo a saída do MGE
TotalCounts, SigmaPixel, qObs = np.loadtxt('MGE_Output_without_arcs.txt', unpack=True)
TotalCounts = TotalCounts*EXPTIME
TotalCounts/TotalCounts[0]

array([1.        , 2.00458658, 1.67272341, 3.49909647, 1.57139447,
       3.92411154, 3.89484221, 7.43209639])

In [9]:
#Agora vamos converter o total de contagens no pico de brilho superfícial C_0
C_0 = TotalCounts/(2*np.pi*(SigmaPixel**2)*qObs) #Via equation (1) Cappellari (2002)
C_0

array([498851.78828564, 154392.0017574 ,  22269.54095846,   7261.54799362,
         4587.33545143,   1880.70170698,   1488.979313  ,    643.60412486])

In [10]:
C_0

array([498851.78828564, 154392.0017574 ,  22269.54095846,   7261.54799362,
         4587.33545143,   1880.70170698,   1488.979313  ,    643.60412486])

In [11]:
SigmaPixel*SCALE

array([0.04569021, 0.13002768, 0.2552859 , 0.58830138, 0.66883275,
       1.22418549, 1.76055489, 3.25551357])

In [12]:
#Agora calculamos o brilho superfícial na banda F160w usando Vega como referência
#Pelo https://www.stsci.edu/itt/review/dhb_2011/WFC3/wfc3_Ch73.html, a relação entre contagens e magnitue é 
                #WFC3MAG = -2.5log (count rate [e− s-1])
#Além disso, a relação entre Surface brightnesses e total magnitude é
                # S = m + 2.5 log(Área)
#em que S é a Surface brightnesses, m a magnitude e Área = Scale² =. Deste modo, obtemos para cada gaussiana do MGE:

S = (-2.5*np.log10(C_0) + 2.5*np.log10(EXPTIME) + 5*np.log10(SCALE) + Vega_zeropoint - A_160)
        #Onde Corrigimos o valor pela extinção galática

In [13]:
#Agora convertemos o surface brightness para surface density I in L_solar/pc².
    #A equação abaixo é encontrada no readme_mge de Cappellari

I = ((64800/np.pi)**2)*(10**(0.4*(M_sun_160 - S)))   
I

array([18346.29269204,  5678.08098531,   819.00782183,   267.0582486 ,
         168.70862417,    69.16664387,    54.76025331,    23.66985532])

In [14]:
#Agora salvamos uma tabela com os dados para o JAM

np.savetxt('JAM_Input.txt', np.column_stack([I, SigmaPixel*SCALE,qObs]),
                            fmt=b'\t%10.6f\t\t\t %10.6f\t %10.6f\t ', 
                            header='\tSurface Density(L_sun/pc²)\t Sigma(arcsec)\t    qObs\t', delimiter='\t')

In [15]:
print('Relative Flux       Scale \tqObs')
for i in range(7):
    print('%10.5f \t%10.6f %10.6f' %(TotalCounts[i]/TotalCounts[0], SigmaPixel[i]*0.04, qObs[i]))

Relative Flux       Scale 	qObs
   1.00000 	  0.020307   0.689692
   2.00459 	  0.057790   0.551571
   1.67272 	  0.113460   0.827813
   3.49910 	  0.261467   1.000000
   1.57139 	  0.297259   0.550000
   3.92411 	  0.544082   1.000000
   3.89484 	  0.782469   0.606143


Referencias:
    

    Vega Zero Point (https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration)
    Willmer, Christopher N. A. (2018) (https://iopscience.iop.org/article/10.3847/1538-4365/aabfdf)
    WFC3 MAG (https://www.stsci.edu/itt/review/dhb_2011/WFC3/wfc3_Ch73.html)